# **Imports &#8595;**

In [2]:
import numpy as np
import pandas as pd 
import os
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
# import torch
# import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import GridSearchCV

c:\Users\metet\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Load Dataset &#8595;**

In [3]:
DATA_PATH = "datasets/eedi-mining-misconceptions-in-mathematics"
EXTERNAL_DATA_PATH = "datasets/eedi-external-dataset"

# train_df = pd.read_csv(f'{DATA_PATH}/train.csv', index_col='QuestionId')
train_df = pd.read_csv(f'{EXTERNAL_DATA_PATH}/all_train.csv', index_col='QuestionId') #this contains the original dataset + an external dataset generated by a LLM
test_df = pd.read_csv(f'{DATA_PATH}/test.csv')
misconceptions_df = pd.read_csv(f'{DATA_PATH}/misconception_mapping.csv')

pd.options.display.max_colwidth = 300
display(train_df.head(5))
pd.options.display.max_colwidth = 50

,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,source,MisconceptionAName,MisconceptionBName,MisconceptionCName,MisconceptionDName,OriginalQuestionId
QuestionId,,,,,,,,,,,,,,,,,,,,
0,856.0,Use the order of operations to carry out calculations involving powers,33.0,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,original,NaN,NaN,NaN,"Confuses the order of operations, believes addition comes before multiplication",NaN
1,1612.0,Simplify an algebraic fraction by factorising the numerator,1077.0,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,original,"Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\n","Thinks that when you cancel identical terms from the numerator and denominator, they just disappear","Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\n",NaN,NaN
2,2774.0,Calculate the range from a list of data,339.0,Range and Interquartile Range from a List of Data,B,"Tom and Katie are discussing the \( 5 \) plants with these heights:\n\( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \)\nTom says if all the plants were cut in half, the range wouldn't change.\nKatie says if all the plants grew by \( 3 \mathrm{~cm} \) each,...",Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,original,Believes if you changed all values by the same proportion the range would not change,NaN,Believes if you changed all values by the same proportion the range would not change,Believes if you add the same value to all numbers in the dataset the range will change,NaN
3,2377.0,Recall and use the intersecting diagonals properties of a rectangle,88.0,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.](),acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,original,Does not know the properties of a rectangle,Does not know the properties of a rectangle,NaN,Does not know the properties of a rectangle,NaN
4,3387.0,Substitute positive integer values into formulae involving powers or roots,67.0,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|}\n\hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\\n\hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\\n\hline\n\end{tabular},\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,original,NaN,NaN,NaN,Thinks you can find missing values in a given table by treating the row as linear and adding on the difference between the first two values given.,NaN


# **Data Preprocessing &#8595;**

In [4]:
def clean(example, columns):
    """
    Cleans the example from the Dataset
    Args:
        example: an example from the Dataset
        columns: columns that will be cleaned

    Returns: update example containing 'clean' columns

    """
    for col in columns:
        text = example[f'{col}']

        # Empty text
        if type(text) not in (str, np.str_) or text=='':
            example[f'clean_{col}'] = ''
            return example

        # 'text' from the example can be of type numpy.str_, let's convert it to a python str
        text = str(text).lower()

        # Clean the text
        text = re.sub("\"", " ", text) # removes the " from certain texts
        text = re.sub("\n", " ", text) # removes the multiple "\n" 
        text = re.sub(r"(\\\w+)(\W)", r" \1 \2", text) # matches with the LaTeX commands like "\hline{}",... and transforms them to " \hline {}"
        text = re.sub(r"([\(|\{|\[|\|])", r" \1", text) # matches every opening parenthesis types and puts spaces on their left
        text = re.sub(r"([\)|\}|\]])", r"\1 ", text) # matches every closing parenthesis types and puts spaces on their right
        text = re.sub(r"\\(?![a-zA-Z])", " ", text) # removes every backslash that is not the start of a LaTeX command
        text = re.sub(r"\( | \)", "", text) # removes the parentheses that appear sometimes from nowhere 
        text = re.sub(r"\[ | \]", "", text) # removes the parentheses that appear sometimes from nowhere
        
        text = re.sub(r" +", " ", text) # cleans the double spaces made by above substitutions
        # Update the example with the cleaned text
        example[f'clean_{col}'] = text.strip()
    return example

# testing_data = {
#     'QuestionText': ["This is a question with a newline\nin the middle"],
#     'AnswerAText': ["Answer A\nwith newline and \\table[test]"],
#     'AnswerBText': ["Answer B\nwith newline and \hline(uwo)"],
#     'AnswerCText': ["Answer C\nwith newline and \color{gold}"],
#     'AnswerDText': ["Answer D\nwith newline and \\begin{tabular}"]
# }
# df = pd.DataFrame(testing_data)
# df = df.apply(clean, axis = 1, columns = columns_to_clean)
# display(df.head(1))

columns_to_clean = ['QuestionText', 'AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText']
train_df = train_df.apply(clean, axis = 1, columns = columns_to_clean)

# Adjust column order
new_order = ['ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'CorrectAnswer']
for col in columns_to_clean:
    new_order.append(col)
    new_order.append(f'clean_{col}')
new_order.extend(['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId', 'source', 'MisconceptionAName', 'MisconceptionBName', 'MisconceptionCName', 'MisconceptionDName', 'OriginalQuestionId'])
train_df = train_df[new_order]


display_train_df = train_df[['QuestionText', 'clean_QuestionText','AnswerAText', 'clean_AnswerAText', 'AnswerBText', 'clean_AnswerBText', 'AnswerCText', 'clean_AnswerCText', 'AnswerDText', 'clean_AnswerDText']]
pd.options.display.max_colwidth = 300
display(display_train_df.head(1))
pd.options.display.max_colwidth = 50

,QuestionText,clean_QuestionText,AnswerAText,clean_AnswerAText,AnswerBText,clean_AnswerBText,AnswerCText,clean_AnswerCText,AnswerDText,clean_AnswerDText
QuestionId,,,,,,,,,,
0,\[\n3 \times 2+4-5\n\]\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,3 \times 2+4-5 where do the brackets need to go to make the answer equal 13 ?,\( 3 \times(2+4)-5 \),3 \times (2+4) -5,\( 3 \times 2+(4-5) \),3 \times 2+ (4-5),\( 3 \times(2+4-5) \),3 \times (2+4-5),Does not need brackets,does not need brackets


# **Reshape Dataset For Training &#8595;**

In [5]:
# train_df columns: QuestionID, ConstructID, ConstructName, CorrectAnswer, SubjectId, SubjectName, QuestionText, Answer[A/B/C/D]Text, Misconception[A/B/C/D]Id

reshaped_data = []
for _, row in train_df.iterrows():
    for answer, misconception_id in zip(
        ['clean_AnswerAText', 'clean_AnswerBText', 'clean_AnswerCText', 'clean_AnswerDText'],
        ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']
    ): # turn the data into a format where each datapoint (row) represents an answer choice (i.e there are now 4 datapoints for each question)
        reshaped_data.append({
            'QuestionText': row['clean_QuestionText'],
            'AnswerText': row[answer],
            'MisconceptionId': row[misconception_id],
            'SubjectName': row['SubjectName'],
            'ConstructName': row['ConstructName']
        })

reshaped_df = pd.DataFrame(reshaped_data)
display(reshaped_df.head())

# removed columns: QuestionId, ConstructId, CorrectAnswer, SubjectId
# other changes: Answer[A/B/C/D]Text are now in separate datapoints along with their associated Misconception[A/B/C/D]Texts 

,QuestionText,AnswerText,MisconceptionId,SubjectName,ConstructName
0,3 \times 2+4-5 where do the brackets need to g...,3 \times (2+4) -5,NaN,BIDMAS,Use the order of operations to carry out calcu...
1,3 \times 2+4-5 where do the brackets need to g...,3 \times 2+ (4-5),NaN,BIDMAS,Use the order of operations to carry out calcu...
2,3 \times 2+4-5 where do the brackets need to g...,3 \times (2+4-5),NaN,BIDMAS,Use the order of operations to carry out calcu...
3,3 \times 2+4-5 where do the brackets need to g...,does not need brackets,1672.0,BIDMAS,Use the order of operations to carry out calcu...
4,"simplify the following, if possible: \frac {m^...",m+1,2142.0,Simplifying Algebraic Fractions,Simplify an algebraic fraction by factorising ...


# **Sentence Embeddings & OneHot Encoding&#8595;**

In [6]:
# remove NaN values (dropping all datapoints that do not have misconceptions assigned to them)
# P.S. that means we are also deleting all the rows (answer choices) that are correct
# P.P.S. unless somehow there are correct answers that have misconceptions associated with them
print(reshaped_df['MisconceptionId'].isnull().sum())  # 10582 NaN values yikes :/
reshaped_df = reshaped_df.dropna(subset=['MisconceptionId'])
print(reshaped_df['MisconceptionId'].isnull().sum())  # 0 now yippie


10582
0


In [12]:
# generate sentence embeddings

# model to test option 1 (0.0023)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# model to test option 2 (0.0015)
# model = SentenceTransformer('math-similarity/Bert-MLM_arXiv-MP-class_zbMath')

def get_sentence_embedding(text):
    return model.encode(text, convert_to_numpy=True)

reshaped_df['CombinedText'] = reshaped_df['QuestionText'] + " " + reshaped_df['AnswerText']
reshaped_df['Embeddings'] = reshaped_df['CombinedText'].apply(get_sentence_embedding)


c:\Users\metet\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\metet\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 

In [8]:
# use One hot encoding for categorical data (create a "column" for each unique subject and construct and represent each row with 0 and 1)
encoder = OneHotEncoder(sparse_output=False)
categorical_features = encoder.fit_transform(reshaped_df[['SubjectName', 'ConstructName']])

# Combine all features
X_embeddings = np.vstack(reshaped_df['Embeddings'].values)
X = np.hstack([X_embeddings, categorical_features])
y = reshaped_df['MisconceptionId']

print(X.shape) # print shape of X to see total number of features 


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42) #Split the data into training and testing


(4370, 1688)


# **Random Forest Training&#8595;**

In [9]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)


# y_pred = rf_classifier.predict(X)
# print(classification_report(y_test, y_pred))


RandomForestClassifier(random_state=42)

# **Testing&#8595;**

In [10]:
def map_at_25(y_true, y_pred_probs, top_k=25):
    
    map_25 = 0.0
    for true_label, pred_prob in zip(y_true, y_pred_probs):
        # Get top_k predictions
        top_preds = np.argsort(pred_prob)[::-1][:top_k]
        
        if not true_label:
            continue
        
        score = 0.0
        hits = 0
        for i, pred in enumerate(top_preds, start=1):
            if pred == true_label:
                hits += 1
                score += hits / i  # Precision at i
        
        # Average Precision at 25
        map_25 += score / min(1, top_k)
    
    return map_25 / len(y_true)


In [11]:
y_val_pred_probs = rf_classifier.predict_proba(X_val)
y_val_true = list(y_val)

map25_score = map_at_25(y_val_true, y_val_pred_probs)
print(f"MAP@25 Score: {map25_score}")


MAP@25 Score: 0.0015414441901856091


In [ ]:
y_pred_probs = rf_classifier.predict_proba(X_val)

# print predictions
for idx, (true_label, pred_prob) in enumerate(zip(y_val, y_pred_probs)):
    # Get top 25 predictions and probabilities
    top_preds = np.argsort(pred_prob)[::-1][:25]
    top_probs = pred_prob[top_preds]
    
    # Check if true is within top 25
    in_top_25 = true_label in top_preds
    
    print(f"Example {idx + 1}")
    print(f"True Label: {true_label}")
    print("Top 25 Predictions (Misconception ID: Probability):")
    for pred, prob in zip(top_preds, top_probs):
        print(f"ID {pred}: {prob:.4f}")
    print(f"True Label in Top 25: {in_top_25}\n")

    # Number of questions to print
    if idx == 10:  
        break

Example 1
True Label: 2030.0
Top 25 Predictions (Misconception ID: Probability):
ID 1052: 0.1000
ID 1025: 0.0600
ID 1138: 0.0600
ID 617: 0.0400
ID 7: 0.0300
ID 1128: 0.0300
ID 1001: 0.0200
ID 1239: 0.0200
ID 1229: 0.0200
ID 381: 0.0200
ID 931: 0.0200
ID 807: 0.0100
ID 973: 0.0100
ID 276: 0.0100
ID 1145: 0.0100
ID 104: 0.0100
ID 283: 0.0100
ID 1315: 0.0100
ID 610: 0.0100
ID 700: 0.0100
ID 588: 0.0100
ID 293: 0.0100
ID 988: 0.0100
ID 1115: 0.0100
ID 74: 0.0100
True Label in Top 25: False

Example 2
True Label: 1794.0
Top 25 Predictions (Misconception ID: Probability):
ID 693: 0.4700
ID 96: 0.1800
ID 404: 0.0800
ID 1065: 0.0100
ID 787: 0.0100
ID 1269: 0.0100
ID 1270: 0.0100
ID 682: 0.0100
ID 494: 0.0100
ID 147: 0.0100
ID 242: 0.0100
ID 248: 0.0100
ID 474: 0.0100
ID 78: 0.0100
ID 735: 0.0100
ID 825: 0.0100
ID 824: 0.0100
ID 440: 0.0100
ID 1351: 0.0100
ID 1149: 0.0100
ID 281: 0.0100
ID 280: 0.0100
ID 95: 0.0100
ID 272: 0.0100
ID 510: 0.0100
True Label in Top 25: False

Example 3
True Label: